# Forcasting the sale price in residential real-estate in Toronto

### Objective
The objective of this notebook is to prepare and explore Toronto freehold housing data, in order to develop a ML model to predict residential housing sale and time on market. Market sales will be used to evaluate future earnings of real-estate mortgage brokers, which will be used in conjunction with human resource accounting (HRA) models for employee valuations [1]. In this notebook, we will prepare the data for ingestion into a Pytorch NN, conduct a brief EDA, and finally train a model.

### Data and Data Collection:
The data was manually scraped from MLS based on previously sold free-hold properties, in the Toronto region from 2006-2018, with housing prices between five-hundred thousand and fifty million. The final scraped dataset contained 49,220 samples, accross 299 features (i.e. address, brokerage, listing price, sale price, number of bedrooms etc.). Once uploaded, the dataset will be reduced for easier computation.

[1] Flamholtz, E. G. (2012). Human resource accounting: Advances in concepts, methods and applications. Springer Science & Business Media.

##### Execution times: 
The execution times for cells are denoted as: `Exec time = ## sec`, and based on the following specs: i5 (7th gen) processors, and 8 Gb ram.

`Exec time = 6.2 sec`

In [ ]:
from __future__ import print_function
import torch
from torch.autograd import Variable
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from scipy.stats import norm
from datetime import date
import os
import time
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline



# Current directory (make sure .csv file is in the CWD)
cwd = os.getcwd()

# Import data
df = pd.read_csv(cwd+'\Toronto_Real_estate_labeled.csv')
print(df.shape)

#Amount of missing data
No_miss = df.isnull().sum().sum()
print('Missing data:' , (No_miss/(df.shape[0]*df.shape[1]))*100, '%')

#Data description
df.describe()
#df.dtypes


### Lets look a little closer at the data.

As you can see there are considerable number of columns that have no value - we will remove those columns. We will also remove any rows that are missing values in `Sold Price` and `Days on Market` since they are our metrics of interest. Likewise, we will also remove columns that have >50% missing values and reduce the size of our dataframe to 5000 rows. We also dropped several columns that were confounding.

`Exec time = 4.3 sec`


In [ ]:
#First lets reduce the frame size so its easier to compute
df = df.sample(n=5000)

# Note: May take time
# Convert all dates using Pandas i.e. contract date, sold date, listing entry date, 
df = df.dropna(axis=1, how='all')
df = df.dropna(subset=['Sold Price'])
df = df.dropna(subset=['Days on Market'])

df = df.dropna(thresh=0.5*len(df), axis=1)
df = df.drop(columns=['#',
                      'star',
                      'shopping',
                      'LSC',
                      'Address',
                      'MLS #',
                      'Address.1',
                      '% Listing Price',
                      'Assessment Roll #',
                      'Area', #All Toronto
                      'Bedrooms Total', #Dulicate
                      'Class', #All residential
                      'Special Designation1',
                      'Co-op Brokerage ', #removed since co-op brokerage is only known at time of sale and not during listing
                      'Commission Co-op Brokerage',
                      'Contact After Expired',
                      'Country', #They were all canada
                      'Directions/Cross Streets', #Were to variable for the dataset
                      'Directload Flag',
                      'Drive',
                      'Duplicate Listing',
                      'Expiry Date',
                      'Extras',
                      'Kitchens Total', #Duplicate
                      'Last Update',
                      'List Brokerage Fax #',
                      'List Brokerage Phone #',
                      ' Listing Entry Date',
                      'Lot Size code',
                      'Municipality', #All toronto
                      'Postal Code', #convert to geo coord later on
                      'Prior LSC',
                      'Province',
                      'Picture Timestamp', 
                      'Timestamp',
                      'Remarks for Broker',
                      'Remarks for Clients',
                      'Sale/Lease', # all sales based on data collection technique
                      'Rooms +', #Missing alot of values
                      'Seller/Lanlord Name',
                      'Seller Property Info Statement',
                      'Sold Entry Date',
                      'Sewers', #Almost all the same
                      'Status', #Almost all unavalible
                      'Tax year',
                      'Water', #Almost all the same
                      'Unavalible Date',
                      'Legal Description: Pt Lt 79 Pl 2170 *',
                      ])

#Convert date feild to panda datetime format
df['Contract Date'] = pd.to_datetime(df['Contract Date'], errors = 'coerce') #if error occurs it generates NaT
df['Sold Date'] = pd.to_datetime(df['Sold Date'], errors = 'coerce')
df['Close Date'] = pd.to_datetime(df['Close Date'], errors = 'coerce')

print(df.shape)
No_miss = df.isnull().sum().sum()
print('Missing data:' , (No_miss/(df.shape[0]*df.shape[1]))*100, '%')

### Lets add some more features

We have immediatly dropped 205 feature and reduced the missing valued from 52% to a managble 10%. Now that we have everything a little more clean, lets bring in some extra features regarding market conditions. The external data comes from the Federal Reserve Bank of St. Louis and provides monthly information on: Housing Price Index, Consumer Price Index, Interbank Rate, Interest Rates, Unemployment Rate, and Working Age Population - all of which we feel will affect real estate sales. 

`Exec time = 2.1 sec`

In [ ]:
# Import market Data
df_CPI = pd.read_csv(cwd+'\Macro data\CPI.csv')
df_HPI = pd.read_csv(cwd+'\Macro data\HPI.csv')
df_BR = pd.read_csv(cwd+'\Macro data\Inter_rate.csv')
df_IR = pd.read_csv(cwd+'\Macro data\Intrest_rates.csv')
df_UR = pd.read_csv(cwd+'\Macro data\\Unemployment_rate.csv')
df_WP = pd.read_csv(cwd+'\Macro data\working_pop.csv')

# Merge the dataset
df_CPI['observation_date'] = pd.to_datetime(df_CPI['observation_date'], errors = 'coerce') #Convert to datetime
df_CPI['Date'] = df_CPI.observation_date.dt.to_period('M') #Convert to Month year

df_HPI['observation_date'] = pd.to_datetime(df_HPI['observation_date'], errors = 'coerce') #Convert to datetime
df_HPI['Date'] = df_HPI.observation_date.dt.to_period('M') #Convert to Month year

df_BR['observation_date'] = pd.to_datetime(df_BR['observation_date'], errors = 'coerce') #Convert to datetime
df_BR['Date'] = df_BR.observation_date.dt.to_period('M') #Convert to Month year

df_IR['observation_date'] = pd.to_datetime(df_IR['observation_date'], errors = 'coerce') #Convert to datetime
df_IR['Date'] = df_IR.observation_date.dt.to_period('M') #Convert to Month year

df_UR['observation_date'] = pd.to_datetime(df_UR['observation_date'], errors = 'coerce') #Convert to datetime
df_UR['Date'] = df_UR.observation_date.dt.to_period('M') #Convert to Month year

df_WP['observation_date'] = pd.to_datetime(df_WP['observation_date'], errors = 'coerce') #Convert to datetime
df_WP['Date'] = df_WP.observation_date.dt.to_period('M') #Convert to Month year


#Add date column to df based on sold date
df['Date'] = df['Sold Date'].dt.to_period('M') #Convert to Month year

#Merge all data frames and drop un-neccessary columns
df = pd.merge(left=df, left_on='Date', right=df_CPI, right_on='Date', how='left')
df = pd.merge(left=df, left_on='Date', right=df_HPI, right_on='Date', how='left')
df = pd.merge(left=df, left_on='Date', right=df_BR, right_on='Date', how='left')
df = pd.merge(left=df, left_on='Date', right=df_IR, right_on='Date', how='left')
df = pd.merge(left=df, left_on='Date', right=df_UR, right_on='Date', how='left')
df = pd.merge(left=df, left_on='Date', right=df_WP, right_on='Date', how='left')

# Now convert the date columns in the dataframe to ordinals
#df['Contract Date'] = df['Contract Date'].apply(date.toordinal)
#df['Sold Date'] = df['Sold Date'].apply(date.toordinal)
#df['Close Date'] = df['Close Date'].apply(date.toordinal)

df = df.drop(columns=['Date',
                     'observation_date_x',
                     'observation_date_y',
                     'observation_date',
                      'Contract Date',
                      'Sold Date',
                      'Close Date',
                     ])
#print(df.columns.to_series().groupby(df.dtypes).groups)
print(df.shape)



### Lets focus on Sold Price and Days on Market

Our working dataset is now ready from some EDA, with 5000 samples across 94 features describing the property and housing market conditions. Now, lets focus on our metrics of interest: `Sold Price` and `Days on Market`, to understand their distributions

`Exec time = 0.4 sec`

In [ ]:
# Histograms
df['Sold Price'].describe()

f, axes = plt.subplots(1, 2)

#histogram
sns.distplot(df['Sold Price'], ax=axes[0]);

#skewness and kurtosis
print("Skewness: %f" % df['Sold Price'].skew())
print("Kurtosis: %f" % df['Sold Price'].kurt())

#histogram
sns.distplot(df['Days on Market'], ax=axes[1]);

#skewness and kurtosis
print("Skewness: %f" % df['Days on Market'].skew())
print("Kurtosis: %f" % df['Days on Market'].kurt())

### What does this tell us?

The results show that both `Sold Price` and `Days on Market` deviate from a normal distribution with a high positive skewness and kurtosis (i.e. Ku >> 3 ). Lets add some constraints to the `Sold price` and `Days on Market` metrics and try to better normalize the data.

`Exec time = 0.48 sec `

In [ ]:
#Constrain and transform the data
df = df[df['Sold Price'] < 1250000]
df = df[df['Days on Market'] < 75]

f, axes = plt.subplots(1, 2)

#Revised histogram and normal probability plot
sns.distplot(df['Sold Price'], fit=norm, ax=axes[0]);
fig = plt.figure()
res = stats.probplot(df['Sold Price'], plot=plt)

#transformed histogram and normal probability plot
sns.distplot(df['Days on Market'], fit=norm, ax=axes[1]);
fig = plt.figure()
res = stats.probplot(df['Days on Market'], plot=plt)

print(df.shape)


### Correlation matrix

By adding the restrictions we reduced many of the outliers while still maintaining a relatively high number of samples. We will work with with this datset for now, but further refinment is needed in next iterations of this model/data. Now lets generate a corrlation matrix of the numerical values.

`Exec time = 1.8 sec `

In [ ]:
#Correlation matrix - ** Drop List Price, Original Price,
corrmat = df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);



### What does it mean?

As expected, the `List Price` and `Sale Price` are highly correlated. Notably, there are several positive and negitive correlations occuring with respect to the market conditions. There are other minor correlations (i.e. `Bedroom No`-`Room No.`etc.), but for the most part, a nice correlation matrix. Again, lets focus on our `Sold Price` and `Days on Market` variables. Lets drill down further and look at the correlation matrices specific our varibles of interest: `Sold Price` and `Days on Market`.

`Exec time = 0.9 sec`

In [ ]:
#Sold Price correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Sold Price')['Sold Price'].index
df_c = df[cols]
cm = df_c.corr()
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

#Days on Market correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Days on Market')['Days on Market'].index
df_c = df[cols]
cm = df_c.corr()
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()




### On to scatter

The specific correlation matrix looks good for both vars wrt their top 10 correlations (tho, the `Original Price`, `Listing Price` and `Sold Price` look slightly problematic). Lets continue and look at some scatter plots for these top correlating vars.


`Exec time = 5.4 sec for both Sold Price and Days on Market` 

In [ ]:
#Sold Price scatterplot
print('Sold Price scatter plot:')
k = 5 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Sold Price')['Sold Price'].index
df_s = df[cols].astype('float64')
df_s.dropna(how='all') #to drop if any value in the row has a nan
sns.set()
sns.pairplot(df_s, diag_kind='kde', size = 2.5)
plt.show();

In [ ]:
print('Days on Market scatter plot:')
#Days on Market scatterplot
k = 5 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Days on Market')['Days on Market'].index
df_s = df[cols].astype('float64')
df_s.dropna(how='all') #to drop if any value in the row has a nan
sns.set()
sns.pairplot(df_s, diag_kind='kde', size = 2.5)
plt.show();



### So whats going on?

Their is some clear trending among `List Price`, `Original Price` and `Sold Price`, and `Composite_HPI`, and realtively little trends with `Days on Market`. Looking at the correlation matrices and scatter plots will be the extent of our EDA. So with being said, lets prepare our data (i.e. one-hot encoding, normalization etc.) for Pytorch. 

`Exec time = 4.8 sec`

In [ ]:
#Reduce dataframe size to 1500 before creating database for training
df = df[0:1500]

#Remove spaces in names
df.columns = df.columns.str.replace('\s+', '_')

#Export current dataset of combined data in its raw form for records
df.to_csv("Toronto_Real_estate_cleaned.csv")

##Un normalized Pytorch export
#df = pd.get_dummies(df)
#df.to_csv("Toronto_Real_estate_cleaned_1hot.csv")

# Normalize all Float and Int datatypes
df_n = df
cols_to_norm = list(df.select_dtypes(exclude=['object']))
df_n[cols_to_norm] = df_n[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

df_n.to_csv("Toronto_Real_estate_ML.csv")

#Rename all vars to remove naming conflicts   
df_n = pd.get_dummies(df_n)
df_n.columns = range(df_n.shape[1])
df_n.to_csv("Toronto_Real_estate_ML_noNames.csv")

print(df.shape)
print(df_n.shape)

### Finally we can play with Pytorch

For familiarization purposes, we will only train a shallow MLP for`Sold Price` prediction, on a small subset of data for reduced computational loading. Futher refinement of the model will take place off-line, and include Days on Market prediction.

`Exec time = 3.2 sec`

In [ ]:
# Import data
df = pd.read_csv(cwd+'\Toronto_Real_estate_ML_noNames.csv')

# Create tensors
train = df
print('Shape of the train data with all features:', train.shape)
train = train.select_dtypes(exclude=['object'])
train.fillna(0,inplace=True)

# Drop Unnamed column from import
train = train.drop(train.columns[train.columns.str.contains('unnamed',case = False)],axis = 1)

#NOTE: Var 1 is Sold_Price and Var 5 is Days on Market 

col_train = list(train.columns)
col_train_bis = list(train.columns)

col_train_bis.remove('1') #1 represents Sold_Price after recoding

mat_train = np.matrix(train)
mat_new = np.matrix(train.drop('1', axis = 1)) #Just training data without sold price
mat_y = np.array(train['1']).reshape((train.shape[0],1)) #Just the sold price by itself

print(mat_new.shape)
print(mat_y.shape)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N = mat_new.shape[0]/50
D_in = mat_new.shape[1] 
H = 10
D_out = 1

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.Tensor(mat_new))
y = Variable(torch.Tensor(mat_y), requires_grad=False)
print(x.shape)
print(y.shape)
# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(100):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

## Conculsion

In this notebook, we generated a clean dataset, performed some preliminary EDA and trained a simple NN model. Although the trained model has not been optimized or used for prediction, the assignment was a great learning exercise, and expanded my understanding of Pytorch (and tensorflow through my readings). This assignment is a valuable stepping stone for expanding my understanding and skills with ML models and theory, and I look forward to building increasingly complex/robust models using these skills.